# The analysis for this thesis was run from commit 3f89dc765e1ec5335a254a7a83f35a4c038e31b6:
https://github.com/statsbomb/open-data/commit/3f89dc765e1ec5335a254a7a83f35a4c038e31b6

From this commit, the dataframes have the following number of entries:

* df_competition: 37 entries
* df_match: 879 entries
* df_lineup: 26794 entries
* df_event: 3198449 entries
* df_freeze: 277829 entries
* df_tactic: 36817 entries
* df_related: 6219794 entries

This notebook takes the StatsBomb json files and turns them into parquet files. These are extremely fast to load so good for this prototyping kind of analysis.

In [1]:
import mplsoccer.statsbomb as sbapi
import pandas as pd
import os
import glob
import numpy as np

# Path
Chosen path where the StatsBomb open-data is located, only processing the new files.

In [2]:
cwd = os.getcwd()
# open data folder is one folder down in the directory. To change if run elsewhere
STATSBOMB_DATA = os.path.join(cwd,'open-data','statsbomb','data')
# save files in folder in current directory. To change if want to save elsewhere
DATA_FOLDER = os.path.join(cwd,'data', 'statsbomb')

# Get the data file paths

In [3]:
event_links = glob.glob(os.path.join(STATSBOMB_DATA, 'events', '**', '*.json'),recursive=True)
lineup_links = glob.glob(os.path.join(STATSBOMB_DATA, 'lineups', '**', '*.json'),recursive=True)
match_links = glob.glob(os.path.join(STATSBOMB_DATA, 'matches', '**', '*.json'),recursive=True)
competition_path = os.path.join(STATSBOMB_DATA, 'competitions.json')

# Make the directory structure

In [4]:
for folder in ['event_raw', 'related_raw', 'freeze_raw', 'tactic_raw', 'lineup_raw']:
    path = os.path.join(DATA_FOLDER, folder)
    if not os.path.exists(path):
        os.mkdir(path)

# Read the competition data

In [5]:
df_competition = sbapi.read_competition(competition_path, warn=False)
# note there is a slight loss of data quality with timestamps, but these aren't relevant for analysis
# pandas has nanoseconds, which aren't supported in parquet (supports milliseconds)
df_competition.to_parquet(os.path.join(DATA_FOLDER, 'competition.parquet'), allow_truncated_timestamps=True)
df_competition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   competition_id      37 non-null     int64         
 1   season_id           37 non-null     int64         
 2   country_name        37 non-null     object        
 3   competition_name    37 non-null     object        
 4   competition_gender  37 non-null     object        
 5   season_name         37 non-null     object        
 6   match_updated       37 non-null     datetime64[ns]
 7   match_available     37 non-null     datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 2.4+ KB


# Keep a copy of the match dataframe

In [6]:
match_path = os.path.join(DATA_FOLDER, 'match.parquet')
if os.path.exists(match_path):
    df_match_copy = pd.read_parquet(match_path).copy()
    update_files = True
else:
    update_files = False

# Read the match data

In [7]:
match_dfs = [sbapi.read_match(file, warn=False) for file in match_links]
df_match = pd.concat(match_dfs)
# again there is a slight loss of quality when saving timestamps, but only relevant for last_updated
df_match.to_parquet(os.path.join(DATA_FOLDER, 'match.parquet'), allow_truncated_timestamps=True)
df_match.info()

Skipping C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\matches\16\42.json: empty json
Skipping C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\matches\16\76.json: empty json
<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 0 to 51
Data columns (total 50 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   match_id                         879 non-null    int64         
 1   match_date                       879 non-null    datetime64[ns]
 2   kick_off                         877 non-null    datetime64[ns]
 3   home_score                       879 non-null    int64         
 4   away_score                       879 non-null    int64         
 5   match_status_360                 878 non-null    object        
 6   last_updated                     879 non-null    datetime64[ns]
 7   last_updated_360                 0 non-null   

# Check which games have been updated

In [8]:
if update_files == True:
    df_match_copy = df_match[['match_id', 'last_updated']].merge(df_match_copy[['match_id', 'last_updated']],
                                                                 how='left', suffixes=['', '_old'], on='match_id')
    df_match_copy = df_match_copy[df_match_copy.last_updated.dt.floor('ms') != df_match_copy.last_updated_old].copy()
    to_update = df_match_copy.match_id.unique()
    
    # get array of event links to update - based on whether they have been updated in the match json
    event_link_ids = np.array([int(os.path.splitext(os.path.basename(link))[0]) for link in event_links])
    event_to_update = [link in to_update for link in event_link_ids]
    event_links = np.array(event_links)[event_to_update]
    
    # get array of lineup links to update - based on whether they have been updated in the match jsons
    lineup_link_ids = np.array([int(os.path.splitext(os.path.basename(link))[0]) for link in lineup_links])
    lineup_to_update = [link in to_update for link in lineup_link_ids]
    lineup_links = np.array(lineup_links)[lineup_to_update]

# Read the lineup data

In [9]:
LINEUP_FOLDER = os.path.join(DATA_FOLDER, 'lineup_raw')
# loop through all the changed links and store as parquet files - small and fast files
for file in lineup_links:
    save_path = f'{os.path.basename(file)[:-4]}parquet'
    try:
        print('Trying:', file)
        df_lineup = sbapi.read_lineup(file, warn=False)
        df_lineup.to_parquet(os.path.join(LINEUP_FOLDER, save_path))
    except:
        print('Skipping:', file)
        pass

Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\lineups\16109.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\lineups\16131.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\lineups\16205.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\lineups\16289.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\lineups\18245.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\lineups\19715.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\lineups\19733.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\lineups\19738.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\lineups\19743.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\lineups\19749.json
Trying: C:

Convert to a single dataframe

In [10]:
# players with split ids
to_replace = {18103: 38522,  # Dietmar Hamann
              17275: 4656,  # Hannah Jayne Blundell
              17524: 4655,  # Jennifer Beattie
              10172: 4644,  # Jill Scott
              4634: 5088,  # Crystal Dunn
              4639: 10395}  # Maren Mjelde

In [11]:
if len(lineup_links) == 0:
    print('No update')
else:
    lineup_files = glob.glob(os.path.join(LINEUP_FOLDER, '*.parquet'))
    df_lineup = pd.concat([pd.read_parquet(file) for file in lineup_files])
    # replace some ids that appear to be split
    df_lineup.player_id.replace(to_replace, inplace=True)
    df_lineup.to_parquet(os.path.join(DATA_FOLDER, 'lineup.parquet'))
    df_lineup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26794 entries, 0 to 35
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   team_id               26794 non-null  int64 
 1   team_name             26794 non-null  object
 2   match_id              26794 non-null  int64 
 3   player_id             26794 non-null  int64 
 4   player_name           26794 non-null  object
 5   player_nickname       14740 non-null  object
 6   player_jersey_number  26794 non-null  int64 
 7   player_country_id     26794 non-null  int64 
 8   player_country_name   26794 non-null  object
dtypes: int64(5), object(4)
memory usage: 2.0+ MB


# Read the event data

In [12]:
# loop through all the changed links and store as parquet files - small and fast files
for file in event_links:
    save_path = f'{os.path.basename(file)[:-4]}parquet'
    try:
        print('Trying:', file)
        dict_event = sbapi.read_event(file, warn=False)
        # save to parquet files
        # using the dictionary key to access the dataframes from the dictionary
        dict_event['event'].to_parquet(os.path.join(DATA_FOLDER, 'event_raw', save_path))
        dict_event['related_event'].to_parquet(os.path.join(DATA_FOLDER, 'related_raw', save_path))
        dict_event['shot_freeze_frame'].to_parquet(os.path.join(DATA_FOLDER, 'freeze_raw', save_path))
        dict_event['tactics_lineup'].to_parquet(os.path.join(DATA_FOLDER, 'tactic_raw', save_path))
    except:
        print('Skipping:', file)
        pass

Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\events\16109.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\events\16131.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\events\16205.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\events\16289.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\events\18245.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\events\19715.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\events\19733.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\events\19738.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\events\19743.json
Trying: C:\Users\sergi77\Desktop\UNIR\TFM\xG_Model_TFM\open-data\statsbomb\data\events\19749.json
Trying: C:\Users\ser

If updating the event dataframe get a list of ids to update

In [13]:
event_files = glob.glob(os.path.join(DATA_FOLDER, 'event_raw', '*.parquet'))
if update_files:
    event_files_id = [int(os.path.splitext(os.path.basename(file))[0]) for file in event_files]
    ids_to_update = [int(os.path.splitext(os.path.basename(link))[0]) for link in event_links]

Function to load old dataframe and combine with updated parquet files

In [14]:
def update(folder, file_type, ids_to_update):
    # get a list of parquet files to add to the old dataframe
    files = glob.glob(os.path.join(folder, f'{file_type}_raw', '*.parquet'))
    files_id = [int(os.path.splitext(os.path.basename(file))[0]) for file in files]
    mask_update = [match_id in ids_to_update for match_id in files_id]
    files = np.array(files)[mask_update]
    # load the old dataframe, filter out changed matches and add the new parquet files
    df_old = pd.read_parquet(os.path.join(folder, f'{file_type}.parquet'))
    df_old = df_old[~df_old.match_id.isin(ids_to_update)]
    df_new = pd.concat([pd.read_parquet(file) for file in files])
    df_old = pd.concat([df_old, df_new])
    return df_old

Single dataframe events

In [15]:
if len(event_links) == 0:
    print('No update')
else:
    if update_files:
        df_event = update(DATA_FOLDER, 'event', ids_to_update)
        df_event.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_event.to_parquet(os.path.join(DATA_FOLDER, 'event.parquet'))
        df_event.info(verbose=True, show_counts=True)        
    else:
        df_event = pd.concat([pd.read_parquet(file) for file in event_files])
        df_event.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_event.to_parquet(os.path.join(DATA_FOLDER, 'event.parquet'))
        df_event.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3198449 entries, 0 to 3268
Data columns (total 95 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   match_id                        3198449 non-null  int64  
 1   id                              3198449 non-null  object 
 2   index                           3198449 non-null  int64  
 3   period                          3198449 non-null  int64  
 4   timestamp_minute                3198449 non-null  int64  
 5   timestamp_second                3198449 non-null  int64  
 6   timestamp_millisecond           3198449 non-null  int64  
 7   minute                          3198449 non-null  int64  
 8   second                          3198449 non-null  int64  
 9   type_id                         3198449 non-null  int64  
 10  type_name                       3198449 non-null  object 
 11  sub_type_id                     280209 non-null   float64
 12  sub

Single dataframe shot freeze frames

In [16]:
if len(event_links) == 0:
    print('No update')
else:
    if update_files:
        df_freeze = update(DATA_FOLDER, 'freeze', ids_to_update)
        df_freeze.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_freeze.to_parquet(os.path.join(DATA_FOLDER, 'freeze.parquet'))
        df_freeze.info(verbose=True, show_counts=True)      
    else:
        freeze_files = glob.glob(os.path.join(DATA_FOLDER, 'freeze_raw', '*.parquet'))
        df_freeze = pd.concat([pd.read_parquet(file) for file in freeze_files])
        df_freeze.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_freeze.to_parquet(os.path.join(DATA_FOLDER, 'freeze.parquet'))
        df_freeze.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277829 entries, 0 to 248
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    277829 non-null  object 
 1   event_freeze_id       277829 non-null  int64  
 2   player_teammate       277829 non-null  bool   
 3   player_id             277829 non-null  int64  
 4   player_name           277829 non-null  object 
 5   player_position_id    277829 non-null  int64  
 6   player_position_name  277829 non-null  object 
 7   x                     277829 non-null  float64
 8   y                     277829 non-null  float64
 9   match_id              277829 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(3)
memory usage: 21.5+ MB


Single dataframe tactics

In [17]:
if len(event_links) == 0:
    print('No update')
else:
    if update_files:
        df_tactic = update(DATA_FOLDER, 'tactic', ids_to_update)
        df_tactic.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_tactic.to_parquet(os.path.join(DATA_FOLDER, 'tactic.parquet'))
        df_tactic.info(verbose=True, show_counts=True)      
    else:
        tactic_files = glob.glob(os.path.join(DATA_FOLDER, 'tactic_raw', '*.parquet'))
        df_tactic = pd.concat([pd.read_parquet(file) for file in tactic_files])
        df_tactic.player_id.replace(to_replace, inplace=True)  # replace some ids that appear to be split
        df_tactic.to_parquet(os.path.join(DATA_FOLDER, 'tactic.parquet'))
        df_tactic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36817 entries, 0 to 43
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    36817 non-null  object
 1   event_tactics_id      36817 non-null  int64 
 2   player_jersey_number  36817 non-null  int64 
 3   player_id             36817 non-null  int64 
 4   player_name           36817 non-null  object
 5   player_position_id    36817 non-null  int64 
 6   player_position_name  36817 non-null  object
 7   match_id              36817 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 2.5+ MB


Single dataframe related events

In [18]:
if len(event_links) == 0:
    print('No update')
else:
    if update_files:
        df_related = update(DATA_FOLDER, 'related', ids_to_update)
        df_related.to_parquet(os.path.join(DATA_FOLDER, 'related.parquet'))
        df_related.info(verbose=True, show_counts=True)     
    else:
        related_files = glob.glob(os.path.join(DATA_FOLDER, 'related_raw', '*.parquet'))
        df_related = pd.concat([pd.read_parquet(file) for file in related_files])
        df_related.to_parquet(os.path.join(DATA_FOLDER, 'related.parquet'))
        df_related.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6219794 entries, 0 to 6303
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   id                 6219794 non-null  object
 1   id_related         6219794 non-null  object
 2   type_name          6219794 non-null  object
 3   index              6219794 non-null  int64 
 4   type_name_related  6219794 non-null  object
 5   index_related      6219794 non-null  int64 
 6   match_id           6219794 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 379.6+ MB
